In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold

import lightgbm as gbm

In [2]:
#read dataset

data = pd.read_csv("/home/jagadeesh/datasets/diabetes.csv")

In [3]:
data.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
trn_x, val_x, trn_y, val_y = train_test_split(data.drop('Outcome',axis=1).values,data.Outcome.values,test_size=0.10)

In [5]:
folds = 3
skf = StratifiedKFold(n_splits=folds, random_state=1)

prediction = np.zeros(len(val_x))

for i, (train_index, test_index) in enumerate(skf.split(trn_x, trn_y)):
    
    trn_dataset = gbm.Dataset(trn_x[train_index],trn_y[train_index])
    val_dataset = gbm.Dataset(trn_x[test_index],trn_y[test_index])

    parameters = {
    "objective": "binary",
     }

    clf = gbm.train(parameters,trn_dataset,valid_sets=val_dataset,early_stopping_rounds=25)
    prediction += clf.predict(val_x)
    
prediction = prediction / folds    

[1]	valid_0's binary_logloss: 0.61752
Training until validation scores don't improve for 25 rounds.
[2]	valid_0's binary_logloss: 0.59093
[3]	valid_0's binary_logloss: 0.571883
[4]	valid_0's binary_logloss: 0.55822
[5]	valid_0's binary_logloss: 0.54569
[6]	valid_0's binary_logloss: 0.534916
[7]	valid_0's binary_logloss: 0.526205
[8]	valid_0's binary_logloss: 0.51858
[9]	valid_0's binary_logloss: 0.510206
[10]	valid_0's binary_logloss: 0.504511
[11]	valid_0's binary_logloss: 0.49658
[12]	valid_0's binary_logloss: 0.493144
[13]	valid_0's binary_logloss: 0.489083
[14]	valid_0's binary_logloss: 0.484834
[15]	valid_0's binary_logloss: 0.481304
[16]	valid_0's binary_logloss: 0.479033
[17]	valid_0's binary_logloss: 0.478646
[18]	valid_0's binary_logloss: 0.479469
[19]	valid_0's binary_logloss: 0.477014
[20]	valid_0's binary_logloss: 0.477478
[21]	valid_0's binary_logloss: 0.479334
[22]	valid_0's binary_logloss: 0.481894
[23]	valid_0's binary_logloss: 0.481994
[24]	valid_0's binary_logloss: 0.

In [6]:
from sklearn.metrics import accuracy_score

rounder = lambda x: 1 if x > 0.51 else 0

predicted = []

for p in clf.predict(val_x):
    predicted.append(rounder(p))
    
accuracy_score(val_y, predicted)

0.7792207792207793